In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# %matplotlib inline
import holoviews as hv
hv.extension('bokeh')
import bokeh
from bokeh.io import output_notebook, show
import bokeh.plotting as bp # figure, show, output_file
from talk_plottingutils import *
output_notebook()

import functools
import toolz

# Cell fate prediction in hematopoietic stem cells
[Prospective identification of hematopoietic lineage choice by deep learning](http://www.nature.com/nmeth/journal/v14/n4/full/nmeth.4182.html)    
Felix Buggenthin, Florian Buettner, Philipp S Hoppe, Max Endele, Manuel Kroiss, Michael Strasser, Michael Schwarzfischer, Dirk Loeffler, Konstantinos D Kokkaliaris, Oliver Hilsenbeck, Timm Schroeder, Fabian J Theis, Carsten Marr<p>
Nature Methods 14, 403–406 (2017)


[Code](https://github.com/QSCD/HematoFatePrediction) 

## Quick recap:
- murine blood stem cells differentiate into distinct cell types
<!--![Orkin1](orkin1.png)-->
![Orkin2](images/orkin2.png)

- continuous in vitro imaging of this process over several days
- can we use cell morphology to predict in advance what kind of type a cell will differentiate into

# Raw data
- brightfield images
- cell tracking

<img src="images/movie_positions.png" alt="Position layout" style="width: 400px;"/>
<img src="images/single_position.png" alt="single position" style="width: 400px;"/>

# Handling the data
a convenient wrapper around the time-lapse data:

    git clone https://github.com/redst4r/movieTools.git

In [ ]:
import config
import imageNormalizer
from movie import Movie
config.TTTDIR = '/home/michi/pythonProjects/deepLearning/Hemato_korea/data/'
movie = Movie('experiment3', verbose=False)  # 140206
movie.get_all_positions()
len(movie.get_all_images())

lets look at a single raw image from position 54, timepoint 3430

In [ ]:
I = movie.loadimage(position=54, timepoint=3430, WL='w00', extension='png', normalizer=imageNormalizer.NoNormalizer())
plot_image_bokeh(I);

In [ ]:
# a couple of consecutive frames
tt = range(3430,3450)
img_list = [movie.loadimage(position=54, timepoint=i, WL='w00', extension='png', normalizer=imageNormalizer.NoNormalizer())[:600:2,:600:2]
        for i in tt]

In [ ]:
hv_original_stack = hv_plot_stack(img_list, tt)

In [ ]:
%%output size=150
%%opts Image style(cmap='Greys_r')
hv_original_stack 

**Observations**
- uneven illumination
- borders of the cover-slip
- dirt

In [ ]:
def image_iterator(movie, position):
    "returns a generator over all images of that position"
    timepoints = [time for (pos, time, wl), fname in movie.get_all_images().items() if pos==position]
    
    timepoints = np.sort(timepoints)
    for t in timepoints:
        yield movie.loadimage(position, timepoint=t, WL='w00', extension='png', normalizer=imageNormalizer.NoNormalizer())      

### mean intensity over time

In [ ]:
# might take 20 sec or so
average_intensity_over_time = [np.mean(I) for I in image_iterator(movie, position=54)]

In [ ]:
plt.plot(average_intensity_over_time)
plt.xlabel('Time')
plt.ylabel('avg. intensity')
plt.show()

### mean image

In [ ]:
Q = toolz.take(100, image_iterator(movie, position=54))  # just average across 100 images
# Q = image_iterator(movie, position=54)  # careful this loads alot of data into RAM
meanI= np.stack(Q).mean(0)

In [ ]:
plot_image_bokeh(meanI);  # plot_image_mpl(meanI); plt.grid(); plt.show()

# Background normalization
**Methods**

1. subtracing the time averaged image (get rid of static dirt)
2. subtract space averaged mean for each image (get rid of changing intensities)
3. more sophisticated: [BaSiC](https://github.com/QSCD/BaSiC). Takes care of all at once
        A BaSiC Tool for Background and Shading Correction of Optical Microscopy Images
        Tingying Peng, Kurt Thorn, Timm Schroeder, Lichao Wang, Fabian J Theis, Carsten Marr, Nassir Navab
        Nature Communication 8:14836 (2017)
    - $I(x,t) = [B(t) \cdot I^{true}(x,t)] \cdot S(x) + D(x)$
        - $S(x)$ flatfield, uneven illumination
        - $D(x)$ darkfield, camera offset, noise
        - $B(t)$ time dependent baseline intensity (i.e. due to photobleaching)
        - smoothness constrains on $S(x),D(x)$ (sparse in Fourier space)

## subtract the time average

In [ ]:
# our test image
I0 = movie.loadimage(position=54, timepoint=2850, WL='w00', extension='png', normalizer= imageNormalizer.NoNormalizer())

In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(1,2,1)
plt.imshow(I0, cmap=plt.cm.Greys_r)
plt.subplot(1,2,2)
plt.imshow(I0-meanI, cmap=plt.cm.Greys_r)
plt.show()

## BaSiC

In [ ]:
I0 = movie.loadimage(position=54, timepoint=2850, WL='w00', extension='png', 
                     normalizer= imageNormalizer.NoNormalizer())

I_basic = movie.loadimage(position=54, timepoint=2850, WL='w00', extension='png', 
                          normalizer= imageNormalizer.BaSiC_Normalizer())

plt.figure(figsize=(20,20))
plt.subplot(1,2,1)
plt.imshow(I0, cmap=plt.cm.Greys_r)
plt.subplot(1,2,2)
plt.imshow(I_basic, cmap=plt.cm.Greys_r)
plt.show()

# Compare to the original frames at the top of the notebook

In [ ]:
# a couple of consecutive frames
tt = range(3430,3450)
img_list_corrected = [movie.loadimage(position=54, timepoint=i, WL='w00', extension='png', normalizer=imageNormalizer.BaSiC_Normalizer())[:600:2,:600:2]
        for i in tt]

In [ ]:
hv_corrected_stack = hv_plot_stack(img_list_corrected, tt)

In [ ]:
%%output size=100
%%opts Image style(cmap='Greys_r')
hv_original_stack + hv_corrected_stack

# A larger timeframe

In [ ]:
tt = range(2830,3430,20)
img_list = [movie.loadimage(position=54, timepoint=i, WL='w00', extension='png', 
                        normalizer=imageNormalizer.BaSiC_Normalizer())[:600:2,:600:2]
        for i in tt]

In [ ]:
%%output size=150
%%opts Image style(cmap='Greys_r')
hv_plot_stack(img_list, tt)